<a href="https://colab.research.google.com/github/csabi0312/modelling-the-age-of-the-oldest-person-in-the-world/blob/main/NM_oldest_person00.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Def loglikelihood

In [ ]:
import numpy as np
import pandas as pd
from scipy.integrate import quad
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import copy
from tqdm import tqdm

def fGGM(x, a, b, c, gamma):
    return (np.exp(-c * x) *
            (1 + (a * (-1 + np.exp(b * x)) * gamma) / b) ** (-((1 + gamma) / gamma)) *
            (c * (b - a * gamma) + a * np.exp(b * x) * (b + c * gamma))) / b

def FGGM(x, a, b, c, gamma):
    return np.exp(-c * x) / (1 + a * gamma * (np.exp(b * x) - 1) / b) ** (1 / gamma)

listofvalues = [
    [1955.8138888888889, 113.17808219178082], [1955.8138888888889, 110.6986301369863],
    [1956.9583333333333, 111.84657534246575], [1956.9583333333333, 107.8958904109589],
    [1958.2472222222223, 109.17808219178082], [1958.2472222222223, 108.56438356164384],
    [1958.7944444444445, 109.11506849315069], [1958.7944444444445, 109.04109589041096],
    [1959.7666666666667, 110.01369863013699], [1959.7666666666667, 109.43561643835616],
    [1959.8972222222221, 109.56712328767124], [1959.8972222222221, 108.46575342465754],
    [1964.9972222222223, 113.56438356164384], [1964.9972222222223, 109.63561643835617],
    [1965.1805555555557, 109.81643835616438], [1965.1805555555557, 109.66301369863014],
    [1965.5972222222222, 110.08219178082192], [1965.5972222222222, 109.57808219178082],
    [1966.025, 110.00821917808219], [1966.025, 109.57260273972602],
    [1968.2222222222222, 111.76712328767124], [1968.2222222222222, 111.17534246575343],
    [1968.4583333333333, 111.41369863013699], [1968.4583333333333, 110.65205479452055],
    [1969.3416666666667, 111.53150684931506], [1969.3416666666667, 111.23835616438356],
    [1970.0277777777778, 111.92876712328767], [1970.0277777777778, 109.4958904109589],
    [1973.1555555555556, 112.62465753424658], [1973.1555555555556, 111.63561643835617],
    [1973.6305555555555, 112.10684931506849], [1973.6305555555555, 110.44109589041096],
    [1973.8333333333333, 110.64383561643835], [1973.8333333333333, 110.59726027397261],
    [1975.4166666666667, 112.17808219178082], [1975.4166666666667, 111.81917808219178],
    [1976.875, 113.28219178082192], [1976.875, 110.38082191780822],
    [1977.9194444444445, 111.42465753424658], [1977.9194444444445, 111.33424657534246],
    [1978.3166666666666, 111.72876712328767], [1978.3166666666666, 111.03013698630137],
    [1981.0583333333334, 113.77534246575343], [1981.0583333333334, 112.05479452054794],
    [1981.1861111111111, 112.17808219178082], [1981.1861111111111, 111.53698630136986],
    [1982.8666666666666, 113.22191780821917], [1982.8666666666666, 112.5068493150685],
    [1983.7833333333333, 113.42191780821918], [1983.7833333333333, 113.26301369863013],
    [1985.125, 114.6082191780822], [1985.125, 113.53972602739726],
    [1986.8055555555557, 115.21643835616439], [1986.8055555555557, 113.3945205479452],
    [1987.0861111111112, 113.67945205479452], [1987.0861111111112, 113.67123287671232],
    [1987.9888888888888, 114.56986301369864], [1987.9888888888888, 114.21369863013699],
    [1988.0277777777778, 114.25479452054795], [1988.0277777777778, 112.88767123287671],
    [1997.5916666666667, 122.44931506849315], [1997.5916666666667, 116.93150684931507],
    [1998.2916666666667, 117.63013698630137], [1998.2916666666667, 117.55890410958904],
    [1999.9972222222223, 119.26575342465753], [1999.9972222222223, 114.14246575342466],
    [2000.8361111111112, 114.98630136986301], [2000.8361111111112, 114.52328767123288],
    [2001.4305555555557, 115.11506849315069], [2001.4305555555557, 114.37260273972603],
    [2002.213888888889, 115.15342465753425], [2002.213888888889, 114.33698630136986],
    [2002.4083333333333, 114.53150684931506], [2002.4083333333333, 114.26849315068493],
    [2002.638888888889, 114.5013698630137], [2002.638888888889, 113.58356164383562],
    [2002.9944444444445, 113.93972602739726], [2002.9944444444445, 113.76986301369863],
    [2003.7416666666666, 114.51780821917808], [2003.7416666666666, 114.37260273972603],
    [2003.8666666666666, 114.4986301369863], [2003.8666666666666, 114.2027397260274],
    [2004.411111111111, 114.74520547945205], [2004.411111111111, 114.7068493150685],
    [2006.6555555555556, 116.95068493150686], [2006.6555555555556, 116.03287671232877],
    [2006.9444444444443, 116.32328767123288], [2006.9444444444443, 115.30684931506849],
    [2007.0638888888889, 115.42739726027398], [2007.0638888888889, 114.17260273972603],
    [2007.075, 114.18356164383562], [2007.075, 114.06575342465753],
    [2007.6166666666666, 114.6054794520548], [2007.6166666666666, 114.31506849315069],
    [2008.9027777777778, 115.6027397260274], [2008.9027777777778, 115.21095890410959],
    [2009.0027777777777, 115.31232876712329], [2009.0027777777777, 114.74246575342465],
    [2009.6944444444443, 115.43287671232876], [2009.6944444444443, 114.33972602739726],
    [2010.3361111111112, 114.97808219178083], [2010.3361111111112, 114.20547945205479],
    [2010.8416666666667, 114.71506849315068], [2010.8416666666667, 114.32328767123288],
    [2011.4722222222222, 114.95068493150686], [2011.4722222222222, 114.81917808219178],
    [2012.925, 116.27397260273973], [2012.925, 115.66849315068494],
    [2012.9611111111112, 115.7041095890411], [2012.9611111111112, 115.66301369863014],
    [2013.4472222222223, 116.14794520547945], [2013.4472222222223, 115.27123287671233],
    [2015.25, 117.07397260273973], [2015.25, 116.74246575342465],
    [2015.263888888889, 116.75616438356164], [2015.263888888889, 115.87123287671233],
    [2015.4611111111112, 116.06849315068493], [2015.4611111111112, 115.94794520547946],
    [2016.3638888888888, 116.85205479452055], [2016.3638888888888, 116.45205479452055],
    [2017.2888888888888, 117.37534246575342], [2017.2888888888888, 117.0986301369863],
    [2017.7055555555555, 117.51780821917808], [2017.7055555555555, 117.11506849315069],
    [2018.3055555555557, 117.71232876712328], [2018.3055555555557, 116.96986301369863]
]

peaks = np.array(listofvalues)[::2]
timespeakslows = np.column_stack((peaks, np.array(listofvalues)[1::2, 1]))

def llh(t, y, z, alpha, KK, b, c, gamma, kappa, CC, birthage):
    def integrand(u):
        return np.exp(-kappa * u) * FGGM(u - birthage, KK * np.exp(-alpha * (t - u + birthage - 2000)), b, c, gamma)

    integral, _ = quad(integrand, z, y, epsabs=1.48e-09, epsrel=1.48e-09)

    return (np.log(fGGM(y - birthage, KK * np.exp(-alpha * (t - y + birthage - 2000)), b, c, gamma)) +
            kappa * (t - z + birthage) + np.log(CC) -
            CC * np.exp(kappa * (t + birthage)) * integral)

def loglikelihood(params):
    alpha, KK, b, gamma = params
    c = 0
    kappa = 0.004987
    CC = 6270
    birthage = 0

    total_llh = sum(
        llh(timespeakslows[i, 0], timespeakslows[i, 1], timespeakslows[i, 2],
            alpha, KK, b, c, gamma, kappa, CC, birthage)
        for i in range(timespeakslows.shape[0])
    )

    final_term = np.log(
        FGGM(
            timespeakslows[-1, 2] - birthage,
            KK * np.exp(-alpha * (timespeakslows[-1, 0] - timespeakslows[-1, 2] + birthage - 2000)),
            b, c, gamma
        )
    )

    return -(total_llh + final_term)




#Optimization

In [ ]:
def optimize_parameters(alpha, KK, b, gamma ,no_of_steps=100, tol_factor=0.0001,min_tol=1e-9, max_tol=0.1, tol_treshold=0.001, tol_dec_factor=0.1, tol_inc_factor=1.1):
    bounds = [(0, 1)] * 4  # Initialize all bounds to None

    # Check if alpha is a float or a tuple with bounds
    if isinstance(alpha, tuple):
        bounds[0] = alpha
        alpha = (alpha[0] + alpha[1]) / 2  # Use the midpoint as the initial value
    else:
        bounds[0] = (alpha, alpha)  # Fixed value, set bounds to the same value

    # Repeat the same process for KK, b, and gamma
    if isinstance(KK, tuple):
        bounds[1] = KK
        KK = (KK[0] + KK[1]) / 2
    else:
        bounds[1] = (KK, KK)

    if isinstance(b, tuple):
        bounds[2] = b
        b = (b[0] + b[1]) / 2
    else:
        bounds[2] = (b, b)

    if isinstance(gamma, tuple):
        bounds[3] = gamma
        gamma = (gamma[0] + gamma[1]) / 2
    else:
        bounds[3] = (gamma, gamma)

    initial_params = [alpha, KK, b, gamma]

    likeli_scores=[]
    for i in tqdm(range(1, no_of_steps+1)):
      result = minimize(loglikelihood, initial_params, method='Nelder-Mead', bounds=bounds, tol=tol_factor)
      optimized_params = result.x
      likeli_scores.append(-loglikelihood(optimized_params))
      initial_params = copy.deepcopy(optimized_params)

      # Calculate percentage improvement
      if i > 1:
        improvement = (likeli_scores[-2] - likeli_scores[-1]) / likeli_scores[-2]
      else:
        improvement = float('inf')

      # Adjust tolerance based on improvement
      if improvement > tol_treshold:  # Example threshold for significant improvement
        tol_factor *= tol_dec_factor  # Decrease tolerance
      else:
        tol_factor *= tol_inc_factor  # Increase tolerance

      # Clamp the tolerance to the minimum value
      tol_factor = min(max(tol_factor, min_tol),max_tol)


    initial_params=list(initial_params)
    initial_params.append(likeli_scores[-1])

    return initial_params

Each parameter (alpha, K, b, gamma) can take two types of values:

* Tuple (e.g., (0,1)): in this case, the parameter value can vary within these bounds during optimization.
* Float number (e.g., 0.3): in this case, the parameter is fixed to this value during optimization.

The function returns the 4 parameter values corresponding to the best log-likelihood value and the log-likelihood itself.






In [ ]:
#Itt az alpha 0 és 0.1 között mozoghat, a KK 0.01 és 0.1 között, a b fixálva van 0.333407-re, a gamma pedig 0.395047-re
optimize_parameters(alpha=(0,0.1), KK=(0.01,0.1), b=0.333407, gamma=0.395047,no_of_steps=20)

100%|██████████| 20/20 [00:01<00:00, 10.29it/s]


[0.0, 0.01, 0.333407, 0.395047, -4317.389983701281]

In [ ]:
optimize_parameters(alpha=(0,.5),KK=(0,.5),b=.09,gamma=(0,.8),no_of_steps=10)

100%|██████████| 10/10 [00:12<00:00,  1.29s/it]


[0.01277338350752401,
 2.950928750281307e-05,
 0.09,
 0.0859561884916345,
 -119.63868488136445]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(.000005,.5),b=.09,gamma=(.05,.8),no_of_steps=10)

100%|██████████| 10/10 [00:13<00:00,  1.32s/it]


[0.012773216064354505,
 2.9510185205030423e-05,
 0.09,
 0.0859569912112169,
 -119.63868486657606]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(.000005,.5),b=.1,gamma=(.05,.8),no_of_steps=10)

100%|██████████| 10/10 [00:15<00:00,  1.51s/it]


[0.014191569101158489,
 1.0551723373966665e-05,
 0.1,
 0.10129985078852546,
 -119.54731396602322]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(.0000005,1),b=.11,gamma=(.05,1),no_of_steps=10)

100%|██████████| 10/10 [00:19<00:00,  1.90s/it]


[0.015610199488666623,
 3.7280050188749485e-06,
 0.11,
 0.11602966510264215,
 -119.47593608236448]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(0,1),b=.12,gamma=(.05,1),no_of_steps=10)

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]


[0.01702900910742269,
 1.3045001718717653e-06,
 0.12,
 0.1302614342795174,
 -119.41951391163859]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(0,2),b=.13,gamma=(.05,2),no_of_steps=10)

100%|██████████| 10/10 [00:19<00:00,  1.93s/it]


[0.01844781693868188,
 4.5293190037208795e-07,
 0.13,
 0.14409000829012059,
 -119.37449725017818]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(0,2),b=.14,gamma=(.05,2),no_of_steps=10)

100%|██████████| 10/10 [00:19<00:00,  1.91s/it]


[0.019866668149227523,
 1.562577151364563e-07,
 0.14,
 0.15758554021945687,
 -119.33831628345906]

In [ ]:
optimize_parameters(alpha=(.005,.5),KK=(0,2),b=.15,gamma=(.05,2),no_of_steps=10)

  0%|          | 0/10 [00:00<?, ?it/s]<ipython-input-1-851fec26fe96>:24: RuntimeWarning: divide by zero encountered in log
  return np.log(fGGM(y - birthage, KK * np.exp(-alpha * (t - y + birthage - 2000)), b, c, gamma)) + kappa * (t - z + birthage) + np.log(CC) - CC * np.exp(kappa * (t + birthage)) * integral
100%|██████████| 10/10 [00:21<00:00,  2.12s/it]


[0.021285535714842936,
 5.362453528660912e-08,
 0.15,
 0.17080690853846014,
 -119.3090681461457]